In [63]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.scientific.sequential import Tokyo_4
from palettable.tableau import GreenOrange_6
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])
# logging.basicConfig(level=logging.DEBUG)

from utils_notebook.utils import (
    ddf, remove_prefix, load_subgraph, remove_keys, camel_to_snake_cols, 
    add_silo_token_name_adjust_precision, 
)
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://graph.node.bean.money/subgraphs/name/beanstalk


In [64]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [65]:
bs.SiloAssetDailySnapshot.timestamp = bs.SiloAssetDailySnapshot.createdAt
q = bs.Query.siloAssetDailySnapshots(
    first=10000, orderBy="season", orderDirection="desc", 
    where={'siloAsset_': {'silo': ADDR_BEANSTALK}}
)
df = sg.query_df([
    q.timestamp, q.depositedBDV, q.siloAsset.token,
])
df.head()
df = remove_prefix(df, "siloAssetDailySnapshots_")
df = remove_prefix(df, 'siloAsset_')
# Remove tokens with internal balances that aren't deposited.
df = df.drop(df[df['depositedBDV'] == 0].index)
df.timestamp = pd.to_datetime(df.timestamp, unit='s')
df = df.sort_values('timestamp').reset_index(drop=True)
add_silo_token_name_adjust_precision(df, 'token', 'depositedBDV')
assert not df.silo_token_name.isna().any()
df.silo_token_name = df.silo_token_name.apply(lambda n: f"bdv_{n}")
df = df.dropna(subset='depositedBDV') 
df = df.drop(columns="token")
df = df.loc[df.depositedBDV > 0]
df.tail()

,timestamp,depositedBDV,silo_token_name
729,2023-02-02,4.083344e+05,bdv_bean_3crv
730,2023-02-03,1.341486e+07,bdv_ur_bean
731,2023-02-03,2.527731e+07,bdv_ur_bean_3crv
732,2023-02-03,4.083344e+05,bdv_bean_3crv
733,2023-02-03,2.107036e+06,bdv_bean


In [66]:
data = df.rename(columns={'depositedBDV': 'value', 'silo_token_name': 'variable'})
data_total = data[['timestamp', 'value']].groupby("timestamp").sum().reset_index()
data_total['variable'] = 'bdv_total'
data = pd.concat([data, data_total]).sort_values("timestamp").reset_index(drop=True)
data.tail(5)
# data_total.head()

,timestamp,value,variable
913,2023-02-03,1.341486e+07,bdv_ur_bean
914,2023-02-03,2.527731e+07,bdv_ur_bean_3crv
915,2023-02-03,4.083344e+05,bdv_bean_3crv
916,2023-02-03,2.107036e+06,bdv_bean
917,2023-02-03,4.120754e+07,bdv_total


In [67]:
lmetrics = data.variable.unique().tolist()
lmetrics.remove('bdv_total')
rmetrics = ['bdv_total'] 
metrics = lmetrics + rmetrics 
colors = {
    m: GreenOrange_6.hex_colors[i] for i, m in enumerate(metrics)
}
chart_area = chart(
    data, 
    'timestamp', 
    lmetrics=lmetrics, 
    lstrategy="stack_area", 
    rmetrics=rmetrics, 
    rstrategy='line',
    yaxis_left_kwargs=dict(title='Bdv', format=".2s"), 
    colors=colors, 
    legend_kwargs=dict(title=None, orient='top'),
    width=400
)
chart_area

alt.LayerChart(...)

In [68]:
# silo asset breakdown 
dmax = data.timestamp.max()
df_last = data.loc[(data.timestamp == dmax) & (data.variable != 'bdv_total')] 
df_pie = pd.DataFrame(
    {
        'category': df_last.variable.values.tolist(), 
        'value': df_last.value.values.tolist(), 
        'label': [
            f"{c}: {int(v):,} \n({v / df_last.value.sum():.1%})" 
            for c, v in zip(df_last.variable.values, df_last.value.values)
        ]
    }
)

chart_pie = (
    alt.Chart(df_pie, title="Asset Breakdown")
    .encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color(
            "category:N", 
            legend=None, 
            scale=alt.Scale(
                domain=lmetrics, 
                range=[colors[m] for m in lmetrics]
            )
        ),
        tooltip="label:N"
    )
    .mark_arc(outerRadius=100, innerRadius=60)
)
chart_pie

alt.Chart(...)

In [69]:
# silo asset breakdown 
dmax = data.timestamp.max()
df_last = data.loc[(data.timestamp == dmax) & (data.variable != 'bdv_total')] 
categories = ['unlocked', 'locked'] 
bdv_total = data.loc[(data.timestamp == dmax) & (data.variable == 'bdv_total')].value.sum()
values = [
    df_last.loc[(df_last.variable == 'bdv_bean') | (df_last.variable == 'bdv_bean_3crv')].value.sum(), 
    df_last.loc[(df_last.variable == 'bdv_ur_bean') | (df_last.variable == 'bdv_ur_bean_3crv')].value.sum(), 
]

df_pie = pd.DataFrame(
    {
        'category': categories, 
        'value': values, 
        'label': [
            f"{v / bdv_total:.1%} {c}"
            for c, v in zip(categories, values)
        ]
    }
)
df_pie.head()

chart_pie_locked = (
    alt.Chart(df_pie, title="Locked Breakdown")
    .encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color(
            "category:N", 
            legend=None, 
            scale=alt.Scale(
                domain=categories, 
                range=[Tokyo_4.hex_colors[2], Tokyo_4.hex_colors[1]]
            )
        ),
        tooltip="label:N"
    )
    .mark_arc(outerRadius=100, innerRadius=60)
)
chart_pie_locked

alt.Chart(...)

In [70]:
c = (
    alt.hconcat(chart_area, chart_pie, chart_pie_locked)
        .resolve_legend(color="independent")
        .resolve_scale(y="independent", color="independent")
        .resolve_axis(y="independent")    
)
css_lines = css_tooltip_timeseries_multi_colored(metrics, colors)
css = '\n'.join(css_lines)
# apply_css(css)
apply_css("")
c

alt.HConcatChart(...)

In [71]:
output_chart(c, css=css)

<IPython.core.display.JSON object>